In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
eda_df = pd.read_csv("/content/drive/MyDrive/DACON_BASIC/eda.csv", index_col=0)
eda_df

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome,ProdTaken
0,1,28.0,Company Invited,1,10.000000,Small Business,Male,3,4.0,Basic,3.0,Married,3.0,0,1,0,1.0,Executive,20384.000000,0
1,2,34.0,Self Enquiry,3,15.524015,Small Business,Female,2,4.0,Deluxe,4.0,Single,1.0,1,5,1,0.0,Manager,19599.000000,1
2,3,45.0,Company Invited,1,15.524015,Salaried,Male,2,3.0,Deluxe,4.0,Married,2.0,0,4,1,0.0,Manager,23624.108895,0
3,4,29.0,Company Invited,1,7.000000,Small Business,Male,3,5.0,Basic,4.0,Married,3.0,0,4,0,1.0,Executive,21274.000000,1
4,5,42.0,Self Enquiry,3,6.000000,Salaried,Male,2,3.0,Deluxe,3.0,Divorced,2.0,0,3,1,0.0,Manager,19907.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1950,1951,28.0,Self Enquiry,1,10.000000,Small Business,Male,3,5.0,Basic,3.0,Single,2.0,0,1,1,2.0,Executive,20723.000000,0
1951,1952,41.0,Self Enquiry,3,8.000000,Salaried,Female,3,3.0,Super Deluxe,5.0,Divorced,1.0,0,5,1,1.0,AVP,31595.000000,0
1952,1953,38.0,Company Invited,3,28.000000,Small Business,Female,3,4.0,Basic,3.0,Divorced,7.0,0,2,1,2.0,Executive,21651.000000,0
1953,1954,28.0,Self Enquiry,3,30.000000,Small Business,Female,3,5.0,Deluxe,3.0,Married,3.0,0,1,1,2.0,Manager,22218.000000,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score

In [ ]:
# 범주형 변수 변환

eda_df = pd.get_dummies(eda_df, drop_first=True)

In [ ]:
x = eda_df.drop(columns=['id', 'ProdTaken'])
y = eda_df[['ProdTaken']]

In [ ]:
from sklearn.preprocessing import StandardScaler

# StandarScaler 적용
scaler = StandardScaler()

scaler.fit(x)
x_scaled = scaler.transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.40, random_state=42)

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import mean_squared_error

ridge = RidgeClassifier().fit(x_train, y_train)
y_pred = ridge.predict(x_test)

print(mean_absolute_error(y_pred, y_test))
print(accuracy_score(y_pred, y_test))

0.17774936061381075
0.8222506393861893


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:1059: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 파라미터 튜닝

Randomsearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = {
    'alpha': [0.01, 0.1, 0.5, 1, 10]
    }

clf = RidgeClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
                               cv = 4, verbose= 1, random_state= 101, n_jobs = -1)
model.fit(x_train,y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:1059: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=4, estimator=RidgeClassifier(), n_jobs=-1,
                   param_distributions={'alpha': [0.01, 0.1, 0.5, 1, 10]},
                   random_state=101, verbose=1)

In [ ]:
print(model.best_params_)
print(model.best_estimator_)

{'alpha': 0.01}
RidgeClassifier(alpha=0.01)


In [ ]:
from sklearn.metrics import classification_report

# print prediction results
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.99      0.90       617
           1       0.80      0.21      0.33       165

    accuracy                           0.82       782
   macro avg       0.81      0.60      0.62       782
weighted avg       0.82      0.82      0.78       782



gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = {
    'alpha': [0.01, 0.1, 0.5, 1, 10]
    }

clf = RidgeClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = grid_search, cv = 4, verbose= 1, random_state= 101, n_jobs = -1)
model.fit(x_train,y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:1059: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=4, estimator=RidgeClassifier(), n_jobs=-1,
                   param_distributions={'alpha': [0.01, 0.1, 0.5, 1, 10]},
                   random_state=101, verbose=1)

In [ ]:
# print best parameter after tuning
print(model.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(model.best_estimator_)

{'alpha': 0.01}
RidgeClassifier(alpha=0.01)


In [ ]:
from sklearn.metrics import classification_report

# print prediction results
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.99      0.90       617
           1       0.80      0.21      0.33       165

    accuracy                           0.82       782
   macro avg       0.81      0.60      0.62       782
weighted avg       0.82      0.82      0.78       782

